In [1]:
from neo4j.v1 import GraphDatabase, basic_auth
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "neo4J"))

In [2]:
#Standard NS: 0
#File NS: 6
#Media NS: -2

#we're going to ignore everything else 
inPage = False
def processPage(title, ft, ID):
    if ft == 0:
        session.run("CREATE (a:article {title: {title}, id:{id}})", {"title": title, "id": ID})
    elif ft == 6: 
        session.run("CREATE (a:file {title: {title}, id:{id}})", {"title": title, "id": ID})
    elif ft == -2:
        session.run("CREATE (a:media {title: {title}, id:{id}})", {"title": title, "id": ID})
session = driver.session() 
with open('enwiki-20170420-pages-articles.xml', 'r') as wiki:
    c = 1
    pc = 0
    title = ""
    ft = None
    ID = None
    isRedirect = None
    for line in wiki:
        line = line.strip()
        if c %  100000 == 0 and not inPage:
            session.close()
            session = driver.session()
        if line == '</page>':
            inPage = False
            c += 1
            if not isRedirect:
                processPage(title, ft, ID)
        elif inPage:
            if pc == 1:
                #Grab Title 
                title = line[7:-8].lower()
                pc += 1
            elif pc == 2:
                #Figure out namespace 
                ft = int(line[4:-5])
                pc += 1
            elif pc == 3:
                ID = int(line[4:-5])
                pc += 1
            elif pc == 4:
                if line[0:9] == "<redirect":
                    isRedirect = True
        elif line == '<page>':
            inPage = True
            isRedirect=False
            pc = 1

In [ ]:
import threading
import time
import logging
import queue
import re
from neo4j.v1 import GraphDatabase, basic_auth

logging.basicConfig(level=logging.WARNING, format='(%(threadName)-9s) %(message)s',)

TEXT_QUEUE_SIZE = 100
DB_QUEUE_SIZE = 21000000

text_queue = queue.Queue(TEXT_QUEUE_SIZE)
db_queue = queue.Queue(DB_QUEUE_SIZE)

class FileReader(threading.Thread):
    def run(self):
        inPage = False
        with open('enwiki-20170420-pages-articles.xml', 'r') as wiki:
            pc = 0
            ft = 0
            ID = 0
            count = 0 
            inText = False
            isRedirect = None
            for line in wiki:
                line = line.strip()
                if line == '</page>':
                    inPage = False
                elif inPage:
                    if pc == 1:
                        pc += 1
                    elif pc == 2:
                        #Figure out namespace 
                        ft = int(line[4:-5])
                        pc += 1
                    elif pc == 3:
                        ID = int(line[4:-5])
                        pc += 1
                    elif pc == 4:
                        if line[0:9] == "<redirect":
                            isRedirect = True
                        pc += 1
                    elif pc > 4:
                        if line[0:5] == '<text':
                            inText = True
                        if not isRedirect and (ft == 0 or ft == 6) and inText:
                            text_queue.put((ft, ID, line), block=True, timeout=None)
                elif line == '<page>':
                    count += 1 
                    inPage = True
                    isRedirect = False
                    inText = False
                    pc = 1
        logging.warning("File Reader Reached End of Control")

class RegexHandler(threading.Thread):
    def run(self):
        empty_count = 0 
        pattern = re.compile("\[\[File\:(\w|\d|\s|\-|\'|_)*\.\w*\|.|\[\[(\w|\d|\s|\-|\||'|\(|\))*\]\]")
        while empty_count < 10:
            entr = None
            try: 
                entr = text_queue.get(block=True, timeout=5)
            except queue.Empty:
                empty_count += 1
                logging.warning("Empty Text Queue Count:" + str(empty_count))
            else:
                empty_count = 0
                search = "MATCH (a:"
                if entr[0] == 0:
                    search = ''.join([search, 'article {id: {id}}), '])
                elif entr[0] == 6: 
                    search = ''.join([search, 'file {id: {id}}), '])
                result = re.finditer(pattern, entr[2])
                for k in result:
                    db_queue.put((entr[1], k.group(0)[2:-2], search), block=True, timeout=None)

class DBTransmitter(threading.Thread):
    def run(self):
        empty_count = 0
        driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "neo4J"))
        session = driver.session()
        session_count = 1
        while empty_count < 20: 
            ID, entr, search = None, None, None
            try:
                ID, entr, search = db_queue.get(timeout = 600)
            except queue.Empty: 
                logging.warning("Empty DB Queue Count:" + str(empty_count))
                empty_count += 1
            else: 
                empty_count = 0
                links = entr.split('|')
                for l in links: 
                    l = l.lower()
                    if l[0:5].lower() == "file:": 
                        session.run(''.join([search, '(b:file {title: {title}}) CREATE \
                        (a)-[r:file_link {source: a.id, target: b.id}]->(b)']), {"id": ID, "title": l})
                    else:
                        session.run(''.join([search, '(b:article {title: {title}}) CREATE \
                        (a)-[r:article_link {source: a.id, target: b.id}]->(b)']), {"id": ID, "title": l})
                    session_count += 1
                if session_count > 100000:
                    logging.WARNING("Opening new session...")
                    session.close()
                    session = driver.session()

file_reader = FileReader(); 
regex_1 = RegexHandler();
regex_2 = RegexHandler(); 
regex_3 = RegexHandler(); 
db_transmit = DBTransmitter(); 
file_reader.start()
time.sleep(2)
regex_1.start()
regex_2.start()
regex_3.start()
time.sleep(2)
db_transmit.start()
file_reader.join()
regex_1.join()
regex_2.join()
regex_3.join()        


(Thread-5 ) Empty Text Queue
(Thread-6 ) Empty Text Queue
